In [ ]:
import re
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import pandas as pd
import seaborn as sns
from pandas.core.groupby.generic import DataFrameGroupBy

plt.style.use('ggplot')
sns.set(rc={'figure.figsize':(40.0, 12.0)})

In [ ]:
df = pd.read_csv("./READ_DESCRIPTION_OF_VOD_Happy_Birthday_________Insert_name_of_choice_here.tsv", delimiter="\t")

In [ ]:
df

In [ ]:
# Get timestamps and floor them to seconds to reduce number of points.
df["timestamp"] = pd.to_datetime(df["timestamp"], format="%H:%M:%S,%f").dt.floor(freq="min")

In [ ]:
df_grp_timestamp = df.groupby("timestamp")

In [ ]:
def pattern_frequency(
    grp_df: DataFrameGroupBy,
    col: str,
    pattern: str,
    pattern_desc: str,
    *,
    ignorecase: bool = False
) -> pd.DataFrame:
    params = {"regex": True}
    if ignorecase:
        params["flags"] = re.IGNORECASE

    return (
        grp_df[col]
        .aggregate(lambda x: x.str.contains(pattern, **params).sum())
        .reset_index()
        .rename(columns={col: "counts"})
        .assign(**{"desc": pattern_desc})
    )

In [ ]:
df_num_frequency = pattern_frequency(df_grp_timestamp, "msg", r"\+2", "num")
df_denom_frequency = pattern_frequency(df_grp_timestamp, "msg", r"-2", "denom")

In [ ]:
df_message_frequency = pd.DataFrame(
    ((grp, len(rows), "total") for grp, rows in df_grp_timestamp.groups.items()),
    columns=["timestamp", "counts", "desc"]
)

In [ ]:
df_message_frequency = pd.concat([
    df_message_frequency,
    df_num_frequency,
    df_denom_frequency
])

In [ ]:
df_message_frequency["timestamp"]

In [ ]:
fig = sns.relplot(
    data=df_message_frequency,
    kind="line",
    x="timestamp",
    y="counts",
    hue="desc"
)
# https://stackoverflow.com/questions/45289482/how-to-plot-int-to-datetime-on-x-axis-using-seaborn
ax = plt.gca()
ax.set_title("Chat Frequency")
label_formatter = ticker.FuncFormatter(
    lambda x, _: pd.to_datetime(x, unit='d').strftime('%H:%M:%S')
)
ax.xaxis.set_major_formatter(label_formatter)
plt.xticks(rotation=50)